In [114]:
import pandas as pd 
pd.options.mode.chained_assignment = None  
import numpy as np 

In [115]:
# Data Paths 
data = '../data/'

eth_historical = data+'eth_historical.pickle'
osqueeth_historical = data+'osqueeth_historical.pickle'

#constants 
funding_period = 420 #days 

In [116]:
eth_historical = pd.read_pickle(eth_historical)
eth_historical = eth_historical.set_index('Date')
eth_historical.head() 

,Contract_Name,Contract_Address,Ticker_Symbol,Currency,Price
Date,,,,,
2022-03-06,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2625.5842
2022-03-05,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2651.4854
2022-03-04,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2841.9568
2022-03-03,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2935.3157
2022-03-02,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2974.9960


In [117]:
osqueeth_historical = pd.read_pickle(osqueeth_historical)
osqueeth_historical = osqueeth_historical.set_index('Date')
osqueeth_historical.head() 

,Contract_Name,Contract_Address,Ticker_Symbol,Currency,Price
Date,,,,,
2022-03-06,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,577.88650
2022-03-05,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,596.35730
2022-03-04,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,643.96970
2022-03-03,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,716.46857
2022-03-02,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,763.19464


In [118]:
historical = eth_historical.join(osqueeth_historical,how='inner',on=['Date'],lsuffix='_left', rsuffix='_right')
historical.head() 

,Contract_Name_left,Contract_Address_left,Ticker_Symbol_left,Currency_left,Price_left,Contract_Name_right,Contract_Address_right,Ticker_Symbol_right,Currency_right,Price_right
Date,,,,,,,,,,
2022-03-06,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2625.5842,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,577.88650
2022-03-05,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2651.4854,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,596.35730
2022-03-04,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2841.9568,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,643.96970
2022-03-03,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2935.3157,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,716.46857
2022-03-02,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USD,2974.9960,Opyn Squeeth,0xf1b99e3e573a1a9c5e6b2ce818b617f0e664e86b,oSQTH,USD,763.19464


Index price: the Squeeth index price is ETH² <br>
Mark price: Mark is the current trading price of Squeeth <br>
implied funding = ln(currentMark / currentIndex) / funding period <br>
const funding_period = 17.5 days 

In [119]:
historical_reduced = historical[['Ticker_Symbol_left','Price_left','Ticker_Symbol_right','Price_right']]
historical_reduced.columns = ['WETH','WETH_PRICE','MARK_TICKER','MARK_TICKER_PRICE']
historical_reduced['INDEX_TICKER'] = 'WETH^2'
historical_reduced['INDEX_TICKER_PRICE'] = np.square(historical_reduced['WETH_PRICE'].to_numpy())
historical_reduced['IMPLIED_FUNDING'] = np.log(historical_reduced['MARK_TICKER_PRICE'].to_numpy(),historical_reduced['INDEX_TICKER_PRICE'].to_numpy())/funding_period
historical_reduced.head() 

,WETH,WETH_PRICE,MARK_TICKER,MARK_TICKER_PRICE,INDEX_TICKER,INDEX_TICKER_PRICE,IMPLIED_FUNDING
Date,,,,,,,
2022-03-06,WETH,2625.5842,oSQTH,577.88650,WETH^2,6.359377,0.015141
2022-03-05,WETH,2651.4854,oSQTH,596.35730,WETH^2,6.390840,0.015216
2022-03-04,WETH,2841.9568,oSQTH,643.96970,WETH^2,6.467652,0.015399
2022-03-03,WETH,2935.3157,oSQTH,716.46857,WETH^2,6.574334,0.015653
2022-03-02,WETH,2974.9960,oSQTH,763.19464,WETH^2,6.637513,0.015804
